<!-- LAB U-Net Model -->
<div class="alert" style="background: linear-gradient(to right, rgb(100,100,100), rgb(200,200,200)); color:white;">

# **U-NET LAB Colorization Model**
***
This notebook implements a U-Net model for image colorization using the LAB color space. The model takes grayscale images (L channel) as input and predicts the a* and b* color channels.

### Key Features:
- LAB color space for better color prediction
- Real-time visualization of colorization progress
- TensorBoard integration for detailed monitoring
- Combined loss function (0.84 MSE + 0.16 MAE)
- Batch normalization and dropout for stability

### Model Architecture:
- Input: Grayscale image (L channel)
- Output: a* and b* channels
- Encoder: 3 levels with increasing filters (64, 128, 256)
- Bottleneck: 512 filters with dropout
- Decoder: 3 levels with skip connections
</div>

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import datetime
import sys

# Add Src directory to path for importing our model
sys.path.append('../Src')
from unet_model_lab import (
    build_unet_lab,
    lab_loss,
    get_callbacks,
    rgb_to_lab
)

## 1. Setup Directories

In [ ]:
# Create directories for logs and models
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join('logs', 'lab_unet_' + timestamp)
model_dir = '../Models'
os.makedirs(log_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

print(f"Logs will be saved to: {log_dir}")
print(f"Models will be saved to: {model_dir}")

## 2. Load and Preprocess Data

In [ ]:
print("Loading training data...")
X_train = np.load("../Data/prepared_data/comic_input_grayscale_train.npy")
y_train = np.load("../Data/prepared_data/comic_output_color_train.npy")
X_test = np.load("../Data/prepared_data/comic_input_grayscale_test.npy")
y_test = np.load("../Data/prepared_data/comic_output_color_test.npy")

print("\nData shapes:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

## 3. Convert to LAB Color Space

In [ ]:
print("Converting to LAB color space...")
y_train_lab = np.array([rgb_to_lab(img) for img in y_train])
y_test_lab = np.array([rgb_to_lab(img) for img in y_test])

# Extract a* and b* channels for training targets
y_train_ab = y_train_lab[:, :, :, 1:]  # Channels 1 and 2 are a* and b*
y_test_ab = y_test_lab[:, :, :, 1:]    # Channels 1 and 2 are a* and b*

# Normalize input images to [-1, 1] range
X_train = (X_train - 0.5) * 2
X_test = (X_test - 0.5) * 2

print("\nProcessed data shapes:")
print(f"X_train: {X_train.shape} (normalized to [-1, 1])")
print(f"y_train_ab: {y_train_ab.shape} (a* and b* channels)")
print(f"X_test: {X_test.shape} (normalized to [-1, 1])")
print(f"y_test_ab: {y_test_ab.shape} (a* and b* channels)")

## 4. Build and Compile Model

In [ ]:
print("Building model...")
input_shape = (256, 256, 1)  # Grayscale input
model = build_unet_lab(input_shape)

# Plot model architecture
tf.keras.utils.plot_model(
    model, 
    to_file=os.path.join(log_dir, 'model_architecture.png'),
    show_shapes=True,
    show_layer_names=True,
    expand_nested=True
)

# Display model summary
model.summary()

In [6]:
# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=lab_loss,
    metrics=['mae', 'mse']
)

## 5. Train Model

In [ ]:
# Get callbacks
callbacks = get_callbacks(
    model_dir=model_dir,
    log_dir=log_dir,
    validation_data=(X_test, y_test)
)

# Train model
print(f"Starting training... Logs will be saved to {log_dir}")
history = model.fit(
    X_train, y_train_ab,
    validation_data=(X_test, y_test_ab),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

## 6. Save Model and Training History

In [ ]:
# Save final model
model.save(os.path.join(model_dir, "final_unet_lab.keras"))

# Plot and save training history
plt.figure(figsize=(12, 4))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# MAE plot
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(log_dir, 'training_history.png'))
plt.show()

print(f"Training completed. Models and visualizations saved in {model_dir} and {log_dir}")

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs